In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-02-24 11:42:39--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  73.9MB/s    in 0.3s    

2025-02-24 11:42:40 (73.9 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [9]:
# add your code here - consider creating a new cell for each section of code
df = pd.merge(df_ratings, df_books, on='isbn')
book_rating_count = df.groupby('title')['rating'].count().reset_index()
book_rating_count.rename(columns={'rating': 'rating_count'}, inplace=True)

# Define popular_books based on a threshold for rating count
popularity_threshold = 50 # Adjust this value as needed
popular_books = book_rating_count[book_rating_count['rating_count'] >= popularity_threshold]['title']

df =df[df["title"].isin(popular_books)]
user_book_rating = df.pivot_table(index=['user'], columns=['title'], values='rating')

# Impute NaN values using the mean for each column
user_book_rating = user_book_rating.fillna(user_book_rating.mean())

user_book_rating_matrix = csr_matrix(user_book_rating)

In [17]:
def get_recommends(book=""):
    model = NearestNeighbors(metric='cosine', algorithm='brute', p=2)
    model.fit(user_book_rating_matrix)

    # Ensure book title is exactly matched
    book = book.strip()

    if book not in user_book_rating.columns:
        print(f"Book '{book}' not found in popular books.")
        return []  # Or handle the error in another way

    index = user_book_rating.columns.get_loc(book)
    distances, indices = model.kneighbors(user_book_rating_matrix[index, :].toarray(), n_neighbors=6)
    recommended_books = [book]

    for i in range(1, len(distances.flatten())):
        recommended_books.append([user_book_rating.columns[indices.flatten()[i]], distances.flatten()[i]])

    return recommended_books


In [24]:
# Check if the book is present in the filtered dataset
book_to_check = "Where the Heart Is"
if book_to_check in user_book_rating.columns:
    print(f"'{book_to_check}' is available for recommendation.")
else:
    print(f"'{book_to_check}' is not available in the filtered dataset. Try using a different title.")


'Where the Heart Is' is available for recommendation.


In [23]:
books = get_recommends("Where the Heart Is")
print(books)



IndexError: index 15599 is out of bounds for axis 0 with size 2444

In [19]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

IndexError: index 7342 is out of bounds for axis 0 with size 2444